# Notebook d'exploration de la table 'edc_prelevements'  

### Objectif  
L'objectif de ce notebook est d'explorer la table 'edc_prelevements' et de trouver les infos clés qui nous permetteront de créer une table intermédiaire pour faciliter la jointure avec 'edc_resultats'.  

En effet, la relation entre les deux tables est une relation N-N. La colonne 'referenceprel' de la table 'edc_prelevements' contient des doublons, car plusieurs UDI peuvent avoir la même reference de prelevement.  

--> Voir documentation sur le sujet : https://outline.services.dataforgood.fr/doc/notes-sur-la-table-edc_prelevements-N7BwMGDZcQ

### 1 - Démonstration du problème rencontré

In [1]:
import duckdb

con = duckdb.connect(database="./../../database/data.duckdb", read_only=True)

In [3]:
# Informations de base sur le tableau edc_prelevements, 2020 - 2024

query = """
SELECT COUNT(*) AS nb_prelevements,
         COUNT(DISTINCT referenceprel) AS nb_prelevements_uniques,
         COUNT(DISTINCT cdreseau) AS nb_reseaux,
         COUNT(DISTINCT cdreseauamont) AS nb_reseaux_amont
FROM edc_prelevements;
"""

informations_prelevements = con.execute(query).fetch_df()
informations_prelevements

,nb_prelevements,nb_prelevements_uniques,nb_reseaux,nb_reseaux_amont
0,2083345,1436114,23767,22897


### Exemple du problème de doublons de prélèvements  

On prend l'année 2024 comme exemple et nous allons regarder spécifiquement ce qu'il se passe sur une référence de prélèvement précise.  

In [7]:
# Exemple "classique" avec relation amont - aval des UDI. On utilise la referenceprel '00100138881' pour l'exemple

con.execute("""
SELECT *
FROM edc_prelevements
WHERE de_partition = 2024
AND referenceprel = '00100138881'
""").fetch_df()

,cddept,cdreseau,inseecommuneprinc,nomcommuneprinc,cdreseauamont,nomreseauamont,pourcentdebit,referenceprel,dateprel,heureprel,conclusionprel,ugelib,distrlib,moalib,plvconformitebacterio,plvconformitechimique,plvconformitereferencebact,plvconformitereferencechim,de_partition,de_ingestion_date
0,001,001000846,01189,INJOUX-GENISSIAT,001000849,INJOUX GENISSIAT,50 %,00100138881,2024-01-29,11h33,Eau d'alimentation conforme aux exigences de q...,CC PAYS BELLEGARDIEN (CCPB),CC TERRE VALSERHONE - REGIE DES EAUX,CC TERRE VALSERHONE - REGIE DES EAUX,C,C,C,C,2024,2025-02-12
1,001,001000847,01189,INJOUX-GENISSIAT,001000849,INJOUX GENISSIAT,34 %,00100138881,2024-01-29,11h33,Eau d'alimentation conforme aux exigences de q...,CC PAYS BELLEGARDIEN (CCPB),CC TERRE VALSERHONE - REGIE DES EAUX,CC TERRE VALSERHONE - REGIE DES EAUX,C,C,C,C,2024,2025-02-12
2,001,001000848,01189,INJOUX-GENISSIAT,001000849,INJOUX GENISSIAT,50 %,00100138881,2024-01-29,11h33,Eau d'alimentation conforme aux exigences de q...,CC PAYS BELLEGARDIEN (CCPB),CC TERRE VALSERHONE - REGIE DES EAUX,CC TERRE VALSERHONE - REGIE DES EAUX,C,C,C,C,2024,2025-02-12
3,001,001000849,01189,INJOUX-GENISSIAT,None,None,None,00100138881,2024-01-29,11h33,Eau d'alimentation conforme aux exigences de q...,CC PAYS BELLEGARDIEN (CCPB),CC TERRE VALSERHONE - REGIE DES EAUX,CC TERRE VALSERHONE - REGIE DES EAUX,C,C,C,C,2024,2025-02-12


Ici, on voit qu'il existe 4 lignes pour 1 prélèvement unique. On remarque que 3 de ces lignes ont le même 'cdreseauamont'. La ligne qui n'a pas de 'cdreseauamont' renseigné est l'UDI en amont sur lequel le prélèvement a été réalisé. Les résultats sont ensuite extrapolés aux autres UDI.  

Maintenant, filtrons la table sur les lignes qui n'ont pas de 'cdreseauamont'. Nous devrions avoir uniquement les prélèvements réalisés sur l'UDI en question.

In [21]:
# On reste sur 2024. Combien de prélèvements sur UDI sans 'cdreseauamont' ?

prelevements_udi_source = con.execute("""
SELECT *
FROM edc_prelevements
WHERE de_partition = 2024
AND cdreseauamont IS NULL
""").fetch_df()

print(
    f"Nombre de lignes de prélèvements : {prelevements_udi_source['referenceprel'].count()}"
)
print(
    f"Nombre de lignes avec 'referenceprel' unique : {prelevements_udi_source['referenceprel'].nunique()}"
)


Nombre de lignes de prélèvements : 217541
Nombre de lignes avec 'referenceprel' unique : 217541


In [19]:
# Nombre de prélèvements uniques en 2024

con.execute(
    """SELECT COUNT(DISTINCT referenceprel) AS nb_referenceprel_unique_2024 FROM edc_prelevements WHERE de_partition = 2024"""
).fetch_df()

,nb_referenceprel_unique_2024
0,287961


Lorsque l'on filtre avec 'cdreseauamont' IS NULL, on obtient uniquement les réseaux sur lesquels les prélèvements sont effectués. Plus de problème de doublons.  

Or, on constate que le nombre de prélèvements uniques en 2024 ne concorde pas (70 420 referenceprel manquants)... Explorons alors les prélèvements uniques 'referenceprel' qui ne sont pas dans la query précédente.

In [23]:
# Trouver le nombre de prélèvements uniques qui ne sont pas dans la query précédente (liste des prélèvements sur UDI "source")

query = """
SELECT DISTINCT referenceprel
FROM edc_prelevements
WHERE referenceprel NOT IN (
    SELECT referenceprel
    FROM edc_prelevements
    WHERE cdreseauamont IS NULL AND de_partition = 2024
) AND de_partition = 2024;
"""

prelevements_udi_non_source = con.execute(query).fetch_df()
prelevements_udi_non_source

,referenceprel
0,00100143631
1,00100143603
2,00100139914
3,00100141771
4,00100139080
...,...
70415,08300289637
70416,08300281784
70417,08300282415
70418,08300284842


Nous retrouvons le nombre exact de lignes manquantes : 70 420.  

Explorons plus en détails ces lignes.

In [25]:
# Montrer un exemple de lignes en doublons qui ont un 'cdreseauamont' qui n'apparaît pas dans la table listant les UDI "source"
# On prend un échantillon

query = """
SELECT *
FROM edc_prelevements
WHERE referenceprel NOT IN (
    SELECT referenceprel
    FROM edc_prelevements
    WHERE cdreseauamont IS NULL AND de_partition = 2024) AND de_partition = 2024
ORDER BY referenceprel
LIMIT 30;
"""

test_query = con.execute(query).fetch_df()
test_query

# Ne garder que les lignes avec doublons sur "referenceprel" via pandas

duplicate_refs = test_query["referenceprel"][
    test_query["referenceprel"].duplicated(keep=False)
]
test_query_duplicates = test_query[test_query["referenceprel"].isin(duplicate_refs)]
test_query_duplicates = test_query_duplicates.sort_values(["referenceprel", "dateprel"])

test_query_duplicates

,cddept,cdreseau,inseecommuneprinc,nomcommuneprinc,cdreseauamont,nomreseauamont,pourcentdebit,referenceprel,dateprel,heureprel,conclusionprel,ugelib,distrlib,moalib,plvconformitebacterio,plvconformitechimique,plvconformitereferencebact,plvconformitereferencechim,de_partition,de_ingestion_date
16,001,001000459,01185,PLATEAU D'HAUTEVILLE,001000458,TTP (CLG) HBA PLAT.HAUTEVILLE LONGECOMBE,100 %,00100139071,2024-01-25,13h26,Eau d'alimentation conforme aux exigences de q...,HT BUGEY AGGLOMERATION REG. HAUTEVI,CA HAUT - BUGEY AGGLOMÉRATION,CA HAUT - BUGEY AGGLOMÉRATION,C,C,C,C,2024,2025-02-12
17,001,001000457,01185,PLATEAU D'HAUTEVILLE,001000458,TTP (CLG) HBA PLAT.HAUTEVILLE LONGECOMBE,100 %,00100139071,2024-01-25,13h26,Eau d'alimentation conforme aux exigences de q...,HT BUGEY AGGLOMERATION REG. HAUTEVI,CA HAUT - BUGEY AGGLOMÉRATION,CA HAUT - BUGEY AGGLOMÉRATION,C,C,C,C,2024,2025-02-12
21,001,001000783,01125,CORVEISSIAT,001000651,TTP (CLG) BSR CORVEISSIAT,100 %,00100139075,2024-01-04,09h06,Eau d'alimentation conforme aux exigences de q...,SI BRESSE SURAN REVERMONT SUEZ,SUEZ EAU FRANCE - ARA,SIE BRESSE SURAN REVERMONT,C,C,C,C,2024,2025-02-12
22,001,001000829,01125,CORVEISSIAT,001000651,TTP (CLG) BSR CORVEISSIAT,None,00100139075,2024-01-04,09h06,Eau d'alimentation conforme aux exigences de q...,SI BRESSE SURAN REVERMONT SUEZ,SUEZ EAU FRANCE - ARA,SIE BRESSE SURAN REVERMONT,C,C,C,C,2024,2025-02-12
23,001,001001171,01125,CORVEISSIAT,001000651,TTP (CLG) BSR CORVEISSIAT,100 %,00100139075,2024-01-04,09h06,Eau d'alimentation conforme aux exigences de q...,SI BRESSE SURAN REVERMONT SUEZ,SUEZ EAU FRANCE - ARA,SIE BRESSE SURAN REVERMONT,C,C,C,C,2024,2025-02-12
24,001,001000831,01125,CORVEISSIAT,001000651,TTP (CLG) BSR CORVEISSIAT,None,00100139075,2024-01-04,09h06,Eau d'alimentation conforme aux exigences de q...,SI BRESSE SURAN REVERMONT SUEZ,SUEZ EAU FRANCE - ARA,SIE BRESSE SURAN REVERMONT,C,C,C,C,2024,2025-02-12
27,001,001000829,01391,SALAVRE,001001721,TTP (L) BSR LIVRAISON ASR,50 %,00100139078,2024-01-30,09h16,Eau d'alimentation conforme aux exigences de q...,SI BRESSE SURAN REVERMONT SOGEDO,SOGEDO LYON,SIE BRESSE SURAN REVERMONT,C,C,C,C,2024,2025-02-12
28,001,001000831,01391,SALAVRE,001001721,TTP (L) BSR LIVRAISON ASR,100 %,00100139078,2024-01-30,09h16,Eau d'alimentation conforme aux exigences de q...,SI BRESSE SURAN REVERMONT SOGEDO,SOGEDO LYON,SIE BRESSE SURAN REVERMONT,C,C,C,C,2024,2025-02-12


Ici, on voit bien plusieurs doublons pour la même référence de prélèvement. Ces doublons ont le même 'cdreseauamont'.  

Ces 'cdreseauamont' ont-ils eux même des lignes si l'on joint edc_prelevements sur 'cdreseau' = 'cdreseauamont' ?

In [29]:
# Est-ce que les cdreseauamont des prélèvements hors UDI "source" sont retrouvables quelque part dans la table edc_prelevements ?

query = """
WITH cte AS (
SELECT cdreseau, cdreseauamont, referenceprel, nomreseauamont
FROM edc_prelevements
WHERE referenceprel NOT IN (
    SELECT referenceprel
    FROM edc_prelevements
    WHERE cdreseauamont IS NULL AND de_partition = 2024) AND de_partition = 2024
ORDER BY referenceprel
)

SELECT COUNT(*) AS nb_udi
FROM cte
JOIN (SELECT * FROM edc_prelevements WHERE de_partition = 2024) p ON cte.cdreseauamont = p.cdreseau
"""

cdreseauamont_cdreseau_2024 = con.execute(query).fetch_df()
cdreseauamont_cdreseau_2024

,nb_udi
0,0


Cela nous confirme bien qu'il existe des lignes qui ont un réseau en amont qui n'est pas retrouvable / qui n'a pas de ligne propre.  

La majorité des lignes n'est pas dans ce cas là, car le réseau en amont dispose bien d'une ligne propre dans la table. Il est possible de faire le lien entre le "cdreseauamont" des UDI en aval et le "cdreseau" des UDI en amont.  

Nous devons désormais essayer de comprendre pourquoi nous avons des lignes avec un 'cdreseauamont' introuvable dans la DB. Est-ce que ces lignes ont quelque chose de spécial ? Est-ce parce que ces prélèvements ont lieu en amont, sur des TTP ou CAP ? Est-ce possible de récupérer les infos des UDI en amont ?

### 2 - Reproduction de l'analyse sur les années 2020 - 2024

In [47]:
# Combien de prélèvements sur UDI sans 'cdreseauamont' ?

years = [2020, 2021, 2022, 2023, 2024]

for y in years:
    query = f"""
    SELECT *
    FROM edc_prelevements
    WHERE de_partition = {y}
    AND cdreseauamont IS NULL
    """
    prelevements_udi_source = con.execute(query).fetch_df()

    print(
        f"({y}) Nombre de lignes de prélèvements avec UDI sans cdreseauamont : {prelevements_udi_source['referenceprel'].count()}"
    )

    print(
        f"({y}) Nombre de lignes avec 'referenceprel' unique : {prelevements_udi_source['referenceprel'].nunique()}"
    )

    print("\n")

(2020) Nombre de lignes de prélèvements avec UDI sans cdreseauamont : 209941
(2020) Nombre de lignes avec 'referenceprel' unique : 209941


(2021) Nombre de lignes de prélèvements avec UDI sans cdreseauamont : 216141
(2021) Nombre de lignes avec 'referenceprel' unique : 216141


(2022) Nombre de lignes de prélèvements avec UDI sans cdreseauamont : 221316
(2022) Nombre de lignes avec 'referenceprel' unique : 221316


(2023) Nombre de lignes de prélèvements avec UDI sans cdreseauamont : 220896
(2023) Nombre de lignes avec 'referenceprel' unique : 220896


(2024) Nombre de lignes de prélèvements avec UDI sans cdreseauamont : 217541
(2024) Nombre de lignes avec 'referenceprel' unique : 217541




In [42]:
# Nombre de prélèvements uniques

for y in years:
    nbprel_unique = con.execute(
        f"""SELECT COUNT(DISTINCT referenceprel) AS nb_referenceprel_unique FROM edc_prelevements WHERE de_partition = {y}"""
    ).fetch_df()

    print(
        f"({y}) Nombre de prélèvements uniques : {nbprel_unique.iloc[0]['nb_referenceprel_unique']}"
    )

    print("\n")

(2020) Nombre de prélèvements uniques : 278588


(2021) Nombre de prélèvements uniques : 285159


(2022) Nombre de prélèvements uniques : 291882


(2023) Nombre de prélèvements uniques : 292525


(2024) Nombre de prélèvements uniques : 287961




In [44]:
# Trouver le nombre de prélèvements uniques qui ne sont pas dans la liste des prélèvements sur UDI "source"

for y in years:
    query = f"""
    SELECT COUNT(DISTINCT referenceprel)
    FROM edc_prelevements
    WHERE referenceprel NOT IN (
        SELECT referenceprel
        FROM edc_prelevements
        WHERE cdreseauamont IS NULL AND de_partition = {y}
    ) AND de_partition = {y};"""

    print(
        f"({y}) Nombre de prélèvements uniques hors UDI source : {con.execute(query).fetchone()[0]}"
    )

    print("\n")


(2020) Nombre de prélèvements uniques hors UDI source : 68647


(2021) Nombre de prélèvements uniques hors UDI source : 69018


(2022) Nombre de prélèvements uniques hors UDI source : 70566


(2023) Nombre de prélèvements uniques hors UDI source : 71629


(2024) Nombre de prélèvements uniques hors UDI source : 70420




In [46]:
# Est-ce que les cdreseauamont des prélèvements hors UDI "source" sont retrouvables quelque part dans la table edc_prelevements ?

for y in years:
    query = f"""
    WITH cte AS (
    SELECT cdreseau, cdreseauamont, referenceprel, nomreseauamont
    FROM edc_prelevements
    WHERE referenceprel NOT IN (
        SELECT referenceprel
        FROM edc_prelevements
        WHERE cdreseauamont IS NULL AND de_partition = {y}) AND de_partition = {y}
    ORDER BY referenceprel
    )

    SELECT COUNT(*) AS nb_udi
    FROM cte
    JOIN (SELECT * FROM edc_prelevements WHERE de_partition = {y}) p ON cte.cdreseauamont = p.cdreseau
    """

    print(
        f"({y}) Nb de cdreseauamont hors UDI source retrouvables dans la table edc_prelevements de la même année : {con.execute(query).fetchone()[0]}"
    )
    print("\n")

(2020) Nb de cdreseauamont hors UDI source retrouvables dans la table edc_prelevements de la même année : 0


(2021) Nb de cdreseauamont hors UDI source retrouvables dans la table edc_prelevements de la même année : 0


(2022) Nb de cdreseauamont hors UDI source retrouvables dans la table edc_prelevements de la même année : 0


(2023) Nb de cdreseauamont hors UDI source retrouvables dans la table edc_prelevements de la même année : 0


(2024) Nb de cdreseauamont hors UDI source retrouvables dans la table edc_prelevements de la même année : 0




### Conclusion de l'analyse sur les années précédentes  

On note les mêmes problèmes, avec plus ou moins les mêmes proportions d'années en années. Chercher l'explication...  

Nous pouvons aussi essayer de voir si les 'cdreseauamont' hors UDI source sur une année apparaissent en 'cdreseau' dans les prélèvements d'autres années. Si c'est le cas, nous pourrons alors résoudre le problème de doublons pour une partie des réseaux...

In [ ]:
# Faire une liste des cdreseauamont hors UDI "source" toutes années confondues

query = """
SELECT DISTINCT cdreseauamont
FROM edc_prelevements
WHERE referenceprel NOT IN (
    SELECT referenceprel
        FROM edc_prelevements
        WHERE cdreseauamont IS NULL
)
ORDER BY cdreseauamont;
"""

cdreseauamont_list = con.execute(query).fetch_df()
cdreseauamont_list

,cdreseauamont
0,001000205
1,001000240
2,001000242
3,001000245
4,001000246
...,...
20417,976000054
20418,976000056
20419,976003465
20420,976003502


In [ ]:
# Création d'une df avec les cdreseau uniques toutes années confondues

cdreseau_list = con.execute(
    """SELECT DISTINCT cdreseau FROM edc_prelevements ORDER BY 1"""
).fetch_df()
cdreseau_list

,cdreseau
0,001000003
1,001000235
2,001000241
3,001000244
4,001000248
...,...
23762,976003487
23763,976003489
23764,976003554
23765,976003881


In [ ]:
# Croisons les deux listes pour voir les réseaux similaires

udi_amont_cdreseau = cdreseau_list.merge(
    cdreseauamont_list, how="outer", left_on="cdreseau", right_on="cdreseauamont"
)

udi_amont_cdreseau

,cdreseau,cdreseauamont
0,001000003,NaN
1,NaN,001000205
2,001000235,NaN
3,NaN,001000240
4,001000241,NaN
...,...,...
44163,NaN,976003502
44164,976003554,NaN
44165,NaN,976003555
44166,976003881,NaN


In [ ]:
# Voir les réseaux qui sont à la fois cdreseauamont et cdreseau

udi_amont_cdreseau[
    udi_amont_cdreseau["cdreseauamont"].notnull()
    & udi_amont_cdreseau["cdreseau"].notnull()
]

,cdreseau,cdreseauamont
58,001000301,001000301
5229,008001235,008001235
5642,009001279,009001279
8271,014000709,014000709
8970,015002401,015002401
9416,015004568,015004568
9548,016000361,016000361
11318,022000045,022000045
11430,022000397,022000397
11471,022000525,022000525


In [113]:
import random

# Voir les réseaux qui sont seulement cdreseauamont

random_index = random.randint(0, 20400)

only_cdreseauamont = udi_amont_cdreseau[
    udi_amont_cdreseau["cdreseauamont"].notnull()
    & udi_amont_cdreseau["cdreseau"].isnull()
]
random_test_df = only_cdreseauamont.reset_index(drop=True)

random_cdreseauamont = random_test_df.iloc[random_index]["cdreseauamont"]
random_cdreseauamont

'067003440'

In [114]:
# Controle détaillé de quelques réseaux ci-dessus

con.execute(
    f""" SELECT * FROM edc_prelevements WHERE cdreseau = '{random_cdreseauamont}' OR cdreseauamont = '{random_cdreseauamont}' """
).fetch_df()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,cddept,cdreseau,inseecommuneprinc,nomcommuneprinc,cdreseauamont,nomreseauamont,pourcentdebit,referenceprel,dateprel,heureprel,conclusionprel,ugelib,distrlib,moalib,plvconformitebacterio,plvconformitechimique,plvconformitereferencebact,plvconformitereferencechim,de_partition,de_ingestion_date
0,067,067001877,67391,REINHARDSMUNSTER,067003440,STATION DE REINHARDSMUNSTER,50 %,06700225305,2020-01-13,12h30,Eau destinée à la consommation humaine conform...,SDEA CL REGION SAVERNE MARMOUTIER,SDEA ALSACE MOSELLE,SDEA ALSACE MOSELLE,C,C,C,N,2020,2025-02-12
1,067,067001877,67391,REINHARDSMUNSTER,067003440,STATION DE REINHARDSMUNSTER,50 %,06700227812,2020-07-09,09h52,Eau destinée à la consommation humaine conform...,SDEA CL REGION SAVERNE MARMOUTIER,SDEA ALSACE MOSELLE,SDEA ALSACE MOSELLE,C,C,C,N,2020,2025-02-12
2,067,067001880,67391,REINHARDSMUNSTER,067003440,STATION DE REINHARDSMUNSTER,95 %,06700225305,2020-01-13,12h30,Eau destinée à la consommation humaine conform...,SDEA CL REGION SAVERNE MARMOUTIER,SDEA ALSACE MOSELLE,SDEA ALSACE MOSELLE,C,C,C,N,2020,2025-02-12
3,067,067001880,67391,REINHARDSMUNSTER,067003440,STATION DE REINHARDSMUNSTER,95 %,06700227812,2020-07-09,09h52,Eau destinée à la consommation humaine conform...,SDEA CL REGION SAVERNE MARMOUTIER,SDEA ALSACE MOSELLE,SDEA ALSACE MOSELLE,C,C,C,N,2020,2025-02-12
4,067,067001877,67391,REINHARDSMUNSTER,067003440,STATION DE REINHARDSMUNSTER,50 %,06700231719,2021-01-14,11h08,Eau destinée à la consommation humaine conform...,SDEA CL REGION SAVERNE MARMOUTIER,SDEA ALSACE MOSELLE,SDEA ALSACE MOSELLE,C,C,C,N,2021,2025-02-12
5,067,067001877,67391,REINHARDSMUNSTER,067003440,STATION DE REINHARDSMUNSTER,50 %,06700235185,2021-07-29,08h55,Eau destinée à la consommation humaine conform...,SDEA CL REGION SAVERNE MARMOUTIER,SDEA ALSACE MOSELLE,SDEA ALSACE MOSELLE,C,C,C,N,2021,2025-02-12
6,067,067001880,67391,REINHARDSMUNSTER,067003440,STATION DE REINHARDSMUNSTER,95 %,06700231719,2021-01-14,11h08,Eau destinée à la consommation humaine conform...,SDEA CL REGION SAVERNE MARMOUTIER,SDEA ALSACE MOSELLE,SDEA ALSACE MOSELLE,C,C,C,N,2021,2025-02-12
7,067,067001880,67391,REINHARDSMUNSTER,067003440,STATION DE REINHARDSMUNSTER,95 %,06700235185,2021-07-29,08h55,Eau destinée à la consommation humaine conform...,SDEA CL REGION SAVERNE MARMOUTIER,SDEA ALSACE MOSELLE,SDEA ALSACE MOSELLE,C,C,C,N,2021,2025-02-12
8,067,067001877,67391,REINHARDSMUNSTER,067003440,STATION DE REINHARDSMUNSTER,50 %,06700239976,2022-02-07,10h56,Eau destinée à la consommation humaine conform...,SDEA CL REGION SAVERNE MARMOUTIER,SDEA ALSACE MOSELLE,SDEA ALSACE MOSELLE,C,C,C,N,2022,2025-02-12
9,067,067001877,67391,REINHARDSMUNSTER,067003440,STATION DE REINHARDSMUNSTER,50 %,06700242985,2022-07-21,08h19,Eau destinée à la consommation humaine conform...,SDEA CL REGION SAVERNE MARMOUTIER,SDEA ALSACE MOSELLE,SDEA ALSACE MOSELLE,C,C,C,N,2022,2025-02-12
